In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
df = pd.read_csv("cust_satisfaction.csv")
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [38]:
df.isnull().sum()
df.dropna(inplace=True)

In [39]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [40]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [41]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal Customer"]

In [42]:
loyal_customer = loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [43]:
balance_df = pd.concat([loyal_customer, disloyal_customer], axis=0)
balance_df.shape

(38905, 12)

In [44]:
cat_col = balance_df.select_dtypes(include=["object"])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
30400,Male,Loyal Customer,Business travel,Business,satisfied
1998,Female,Loyal Customer,Business travel,Business,satisfied
74831,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
76655,Male,Loyal Customer,Business travel,Business,satisfied
91064,Male,Loyal Customer,Business travel,Business,satisfied


In [45]:
num_col = balance_df.select_dtypes(exclude=["object"])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
30400,46,997,5,5,3,3,48.0
1998,28,1626,4,1,4,0,0.0
74831,23,2026,4,3,4,0,0.0
76655,42,3251,5,5,3,51,75.0
91064,64,1699,4,4,5,1,0.0


In [46]:
pd.get_dummies(cat_col, drop_first=True).astype(int).head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
30400,1,0,0,0,0,1
1998,0,0,0,0,0,1
74831,0,0,1,1,0,0
76655,1,0,0,0,0,1
91064,1,0,0,0,0,1


In [47]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe = OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [48]:
column_name = list(ohe.get_feature_names_out())
column_name

['Gender_Male',
 'Customer Type_disloyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [49]:
one_hot = pd.DataFrame(cat_col_encoded, columns=column_name)
one_hot.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [50]:
one_hot = one_hot.reset_index(drop=True)
num_col = num_col.reset_index(drop=True)
final_df = pd.concat([one_hot, num_col], axis=1)
final_df.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,46,997,5,5,3,3,48.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,28,1626,4,1,4,0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,23,2026,4,3,4,0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,42,3251,5,5,3,51,75.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,64,1699,4,4,5,1,0.0


In [51]:
##x and y ---> train test split -->algo train
x = final_df.drop("Customer Type_disloyal Customer", axis=1)
y = final_df["Customer Type_disloyal Customer"]

In [52]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [53]:
##deep learning -- tensorflow keras
# pip install tensorflow

In [54]:
##standard scaler 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [56]:
x_train.shape[1]

13

In [57]:
##define your ann model 
model = Sequential()
##input layer 
##68 -- no of neurons -- general convention -- 128 
model.add(Dense(68,activation="relu",input_dim = x_train_scaled.shape[1]))
##hidden layer 
model.add(Dense(32, activation="relu"))
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
##output layer 
model.add(Dense(1, activation="sigmoid"))
##model compile 
model.compile(optimizer="adam",
               loss="binary_crossentropy",
                 metrics=["accuracy"])
model.summary()

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 68)             │           952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,265 (16.66 KB)

 Trainable params: 4,265 (16.66 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# (13+1)* 68

# (68+1) *32

In [59]:
history = model.fit(x_train_scaled,y_train,
                    epochs = 10 ,
                    validation_data=(x_test_scaled,y_test))

Epoch 1/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8525 - loss: 0.3409 - val_accuracy: 0.9070 - val_loss: 0.2347
Epoch 2/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9106 - loss: 0.2294 - val_accuracy: 0.9153 - val_loss: 0.2198
Epoch 3/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9174 - loss: 0.2147 - val_accuracy: 0.9144 - val_loss: 0.2159
Epoch 4/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9191 - loss: 0.2135 - val_accuracy: 0.9195 - val_loss: 0.2092
Epoch 5/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9194 - loss: 0.2102 - val_accuracy: 0.9125 - val_loss: 0.2203
Epoch 6/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9220 - loss: 0.2056 - val_accuracy: 0.9199 - val_loss: 0.2077
Epoch 7/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9263 - loss: 0.1958 - val_accuracy: 0.9203 - val_loss: 0.2015
Epoch 8/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9244 - loss: 0.1957 - val_accuracy: 0.

In [60]:
## prediction 
y_pred = model.predict(x_test_scaled)
prediction_label = (y_pred > 0.5).astype(int).ravel()
prediction_label

244/244 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


array([0, 1, 1, ..., 1, 0, 0])

In [61]:
model.save("model_classfication.h5")

In [62]:
from tensorflow.keras.models import load_model
model_load = load_model("model_classfication.h5")